오늘은, 노래가사를 input data로 이용하여, 작사를 하는 AI인공지능을 만들어 보았습니다.

제가 사용한 노래는 Adele, Beatles, Bieber, Britney-spears, Bruno Mars, Disney,michael-jackson, rihanna, paul-smith의 노래입니다.

제공받은 노래의 종류는 더 많지만 그것을 전부 사용하지 않은 이유는 다음과 같습니다.

1. 전체 노래를 학습시키기에는 양이 너무 많다. 학습시킬 데이터 양이 많으면 시간이 오래걸릴거라 생각하여 선별했습니다.
2. 전처리를 위해서는 데이터가 어떤 형식인지 대강이라도 먼저 살펴보아야 하는데, 그 모든 노래를 살펴볼수 없었다.
    - 너무 반복되는 문장을 삭제하기 위함입니다.
3. 제가 아는 가수 위주로 골랐습니다.
    - 이 이유는, 노래의 분위기를 맞춰주려고 한 것입니다. 예를 들어 신문기사를 쓰는 AI를 작성하려는데, 아동의 동요 데이터도 같이 학습 시킨다면, 이는 노이즈 데이터라고 생각합니다.
    - 이와 같은 이유로, eminem가수를 저는 알지만, disney노래와는 잘 맞지 않는 분위기입니다. 그래서 제외시켰습니다.

In [50]:
import glob
import os
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split

txt_file_path = os.getenv('HOME')+'/aiffel/Aiffel_Exploration_nodes/lyricist/mydata/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 34896
Examples:
 ['Looking for some education', 'Made my way into the night', 'All that bullshit conversation']


# 1. 데이터 전처리하기


In [51]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#        strip문자의 특성
#     2. 특수문자는 삭제해줍니다.
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,'¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r"", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?\'|.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence


삭제 리스트 
데이터 셋을 훑어보고 5회이상 같은 가사가 반복되면 삭제시켰습니다.
각종 특수문자가 포함된 문장은 삭제시켰습니다.

In [52]:
# 여기에 정제된 문장을 모을겁니다
corpus = []


for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence.find(":")==1: continue
    if sentence.find("]")==1: continue
    if sentence.find(")")==1: continue
    if sentence.find("*")==1: continue
    #if sentence.find("-")==1: continue
    """
    if sentence.find("ooo")==1: continue
    if sentence.find("We are the new America")==1: 
        continue
    if sentence.find("This is the new America")==1: 
        continue
    if sentence.find("American, American oxygen American oxygen")==1: 
        continue
    if sentence.find("Baby I love you")==1: 
        continue
    if sentence.find("Why You Wanna Trip On Me")==1: 
        continue    
    if sentence.find("Baby don't make me")==1: 
        continue    
    if sentence.find("He really thought he really had")==1: 
        continue    
    if sentence.find("They thought they really had control of me")==1: 
        continue            
    if sentence.find("Dom Sheldon is a cold man")==1: 
        continue    
    if sentence.find("Where Did She go")==1: 
        continue            
    if sentence.find("Im Looking For The Girl")==1: 
        continue    
    if sentence.find("Gimme")==1: 
        continue    
    if sentence.find("Not alone not alone not alone")==1: 
        continue            
    if sentence.find("You're fillin me up")==1: 
        continue    
    if sentence.find("baby, baby,")==1: 
        continue
    if sentence.find("Get naked")==1: 
        continue
    if sentence.find("oh, oh")==1: 
        continue
    if sentence.find("What do you mean?")==1: 
        continue
    if sentence.find("That should be me")==1: 
        continue
    if sentence.find("Madly in love")==1: 
        continue
    if sentence.find("I'll show you")==1: 
        continue
    if sentence.find("I smile")==1: 
        continue
    if sentence.find("Nothing's gonna change my world")==1: 
        continue
    if sentence.find("Nah nah nah nah nah nah, nah nah nah, hey Jude")==1: 
        continue
    if sentence.find("Sun, sun, sun, here it comes")==1: 
        continue
    if sentence.find("I want to hold your hand")==1: 
        continue
    if sentence.find("Come together, yeah")==1: 
        continue
    if sentence.find("All together now")==1: 
        continue
    if sentence.find("One, two, three, four, five, six, seven")==1: 
        continue
    if sentence.find("All good children go to heaven")==1: 
        continue
    if sentence.find("We're on our way home")==1: 
        continue
    if sentence.find("Rumor has it, ooh")==1: 
        continue
    if sentence.find("Say it ain't so, say it ain't so")==1: 
        continue
    if sentence.find("We will stand tall")==1: 
        continue
    if sentence.find("When it crumbles")==1: 
        continue
    if sentence.find("I miss you, I miss you")==1: 
        continue
    if sentence.find("Why do you love me")==1: 
        continue
    """                 
        
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)

print(len(corpus))
corpus[:30]

    

33204


['<start> looking for some education <end>',
 '<start> made my way into the night <end>',
 '<start> all that bullshit conversation <end>',
 "<start> baby can't you read the signs i won't bore you with the details baby <end>",
 "<start> i don't even wanna waste your time <end>",
 "<start> let's just say that maybe <end>",
 '<start> you could help me ease my mind <end>',
 "<start> i ain't mr right but if you're looking for fast love <end>",
 "<start> if that's love in your eyes <end>",
 "<start> it's more than enough <end>",
 '<start> had some bad love <end>',
 "<start> so fast love is all that i've got on my mind ooh ooh <end>",
 '<start> ooh ooh looking for some affirmation <end>',
 '<start> made my way into the sun <end>',
 '<start> my friends got their ladies <end>',
 "<start> and they're all having babies <end>",
 "<start> i just wanna have some fun i won't bore you with the details baby <end>",
 "<start> i don't even wanna waste your time <end>",
 "<start> let's just say that maybe

In [53]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=10000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2 226  25 ...   0   0   0]
 [  2 283  12 ...   0   0   0]
 [  2  22  15 ...   0   0   0]
 ...
 [  2  17  17 ...   0   0   0]
 [  2  18  22 ...   0   0   0]
 [  2  17  17 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f1699462e20>


In [54]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 50: break
#어떻게 구축되었는지.        

1 : <unk>
2 : <start>
3 : <end>
4 : you
5 : i
6 : the
7 : me
8 : to
9 : and
10 : it
11 : a
12 : my
13 : in
14 : love
15 : that
16 : your
17 : oh
18 : i'm
19 : on
20 : be
21 : don't
22 : all
23 : of
24 : is
25 : for
26 : like
27 : so
28 : know
29 : baby
30 : yeah
31 : just
32 : we
33 : but
34 : no
35 : what
36 : do
37 : with
38 : it's
39 : you're
40 : got
41 : up
42 : when
43 : can
44 : get
45 : now
46 : if
47 : one
48 : girl
49 : this
50 : go


In [55]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    
print(len(src_input))
print(src_input[9])
print(tgt_input[9])

33204
[  2  38  87 202 203   3   0   0   0   0   0   0   0   0]
[ 38  87 202 203   3   0   0   0   0   0   0   0   0   0]


In [56]:
# 평가 데이터셋 분리

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                    tgt_input, 
                                                    test_size=0.3)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (23242, 14)
Target Train: (23242, 14)


In [57]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE
print(BUFFER_SIZE, steps_per_epoch)

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

33204 129


<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [58]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.drop1 = tf.keras.layers.Dropout(0.3)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.drop2 = tf.keras.layers.Dropout(0.3)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.drop1(out)
        out = self.rnn_2(out)
        out = self.drop2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [59]:
    
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1):break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 10001), dtype=float32, numpy=
array([[[-1.34137459e-04, -2.55208317e-04, -3.03804482e-05, ...,
          2.88460084e-04,  2.41612273e-04, -1.80752322e-05],
        [-2.90620519e-04, -3.90627771e-04, -1.74332672e-04, ...,
          3.33910139e-04,  1.36578383e-04,  1.95199376e-04],
        [-4.42413148e-04, -6.70602429e-04, -2.33741201e-04, ...,
          6.02601736e-04,  2.63920781e-04,  3.12030897e-04],
        ...,
        [ 5.72616991e-05,  2.45018862e-04,  1.15367572e-03, ...,
          1.82833034e-03,  7.26334460e-04, -1.89103733e-03],
        [-7.65394288e-05,  2.10251485e-04,  1.25895196e-03, ...,
          1.66818290e-03,  3.30021780e-04, -2.05096230e-03],
        [-2.44880765e-04,  1.97050816e-04,  1.37892750e-03, ...,
          1.48230547e-03, -1.09067318e-04, -2.21078936e-03]],

       [[ 2.23472976e-04,  2.00384711e-05,  4.39451454e-04, ...,
         -8.95697740e-05,  2.85182963e-04,  1.73897090e-04],
        [ 1.71744832e-04,  6.01300744e-05,  6

In [60]:
model.summary()

Model: "text_generator_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      multiple                  2560256   
_________________________________________________________________
lstm_4 (LSTM)                multiple                  5246976   
_________________________________________________________________
dropout_3 (Dropout)          multiple                  0         
_________________________________________________________________
lstm_5 (LSTM)                multiple                  8392704   
_________________________________________________________________
dropout_4 (Dropout)          multiple                  0         
_________________________________________________________________
dense_2 (Dense)              multiple                  10251025  
Total params: 26,450,961
Trainable params: 26,450,961
Non-trainable params: 0
______________________________________

In [61]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
#model.fit(dataset, epochs=30)
model.fit(enc_train, dec_train, validation_data=(enc_val, dec_val),epochs=20)


Epoch 1/20
727/727 [==============================] - 27s 34ms/step - loss: 3.2347 - val_loss: 2.9711
Epoch 2/20
727/727 [==============================] - 24s 33ms/step - loss: 2.8370 - val_loss: 2.7539
Epoch 3/20
727/727 [==============================] - 25s 34ms/step - loss: 2.6124 - val_loss: 2.5927
Epoch 4/20
727/727 [==============================] - 25s 34ms/step - loss: 2.4138 - val_loss: 2.4706
Epoch 5/20
727/727 [==============================] - 25s 35ms/step - loss: 2.2274 - val_loss: 2.3765
Epoch 6/20
727/727 [==============================] - 25s 35ms/step - loss: 2.0493 - val_loss: 2.3012
Epoch 7/20
727/727 [==============================] - 25s 35ms/step - loss: 1.8845 - val_loss: 2.2457
Epoch 8/20
727/727 [==============================] - 25s 35ms/step - loss: 1.7348 - val_loss: 2.1999
Epoch 9/20
727/727 [==============================] - 26s 35ms/step - loss: 1.6016 - val_loss: 2.1767
Epoch 10/20
727/727 [==============================] - 26s 35ms/step - loss: 1.488

In [62]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [63]:
generate_text(model, tokenizer, init_sentence="<start> if")

'<start> if you want the one you love <end> '

In [64]:
generate_text(model, tokenizer, init_sentence="<start> l")

'<start> l t b rap performance <end> '

In [65]:
generate_text(model, tokenizer, init_sentence="<start> i")

"<start> i don't know why you say goodbye i say hello wow oh hello <end> "

In [66]:
generate_text(model, tokenizer, init_sentence="<start> how")

'<start> how does it feel <end> '

In [67]:
generate_text(model, tokenizer, init_sentence="<start> a")

'<start> a whale of a tale or two <end> '

In [68]:
generate_text(model, tokenizer, init_sentence="<start> what")

'<start> what about us <end> '

In [69]:
generate_text(model, tokenizer, init_sentence="<start> you")

'<start> you know i love you <end> '

In [70]:
generate_text(model, tokenizer, init_sentence="<start> i")

"<start> i don't know why you say goodbye i say hello wow oh hello <end> "

In [71]:
generate_text(model, tokenizer, init_sentence="<start> i can't")

"<start> i can't let her get away <end> "

In [72]:
generate_text(model, tokenizer, init_sentence="<start> i feel")

'<start> i feel so alone <end> '

# 회고

nlp에 대해서는 처음 접해봐서 이 프로젝트가 전반적으로 어떻게 돌아가는지 이해가 되지 않아 어려움이 많았다. 하지만, 다행히 MO스터디에서 도움을 받아 프로젝트를 끝마칠수는 있었다. ~~(실은, 끝내기 조차 못할뻔 했던,,,)~~

아쉬웠던 점이라면, NLP모델에 대해선 미리 사전학습 해본적이 없어서, 기존 모델을 사용할 수 밖에 없었고, 현 상황에서 더 할 수 있는 것은, 데이터를 좀 더 가려서 넣어주는 것 뿐이었던 점이 아쉬웠다. 시간이 더 생긴다면, nlp 모델을 더 다양하게 써보고 싶고, 앙상블과 스태킹이라는 기법도 사용하여서 성능을 안정화 시켜보고싶다.

학습시키다가 실수로 지워버렸지만, 처음 NLP의 결과물이 i로 시작하는 문장을 만들었을 때, i로 시작하는 문장을 작문했을 때, 당신이 날 얼마나 사랑하는지 느끼며 살아간다. 라는 뉘앙스로 작문이 되었었는데, ~~너무 설렜는데~~실수로 전체 셀을 지워버려서(T_T)
결과를 날려서 너무 아쉽다.

NLP실습을 하면서 제일 신기했던 것은, 데이터 전처리를 많이 해줄수록 오히려 이상한 문장을 뱉어낸다는 것이다. 처음에 전처리할 때는 같은 문장이 5번 이상 반복되면 단어를 작문할때 확률이 그 문장에 대해 비중이 커질 것같아서, 중복된 문장을 전부 찾아서 제거해주었었다. 그러면 더 다양한 문장을 생성해줄거라고 생각했는데, 오히려 더 반복되는 중복적인 문장을 뱉어내었다. 구조를 이해를 잘못한건지,, 궁금하고 공부를 더 해야겠다고 생각했다.
또한,, 에퐄을 더 많이 돌릴수록,, 마지막 부분에 val_loss가 커졌다. 오버핏이 된상황인건가,, 어떻게 된건지 궁금하다. 
NLP는 많은 공부가 더필요할듯 싶다ㅠ_ㅠ!